贝叶斯分类器的分类原理：通过某对象的先验概率，利用贝叶斯公式计算出其后验概率，即该对象属于某一类的概率，选择具有最大后验概率的类作为该对象所属的类。

主要特点：
1. 属性可以离散，也可以连续
2. 数学基础扎实，分类效果稳定
3. 对确实和噪声数据不太敏感
4. 属性如果不相关，分类效果很好；如果相关，则不低于决策树。

- $P(Y)$为先验概率，是根据以往的数据分析或者经验得到的概率。
- $P(Y|X)$为后验概率，是得到本次试验的信息从而重新修正的概率。

**全概率公式**：设试验$E$的样本空间为$S$，$A$为$E$的事件，$B_1,B_2,\cdots, B_n$为样本空间$S$的一个划分，且$P(B_i)\geq 0 (i=1,2,\cdots,n)$，则有：
$$P(A)=P(A|B_1)P(B_1)+P(A|B_2)P(B_2)+\cdots+P(A|B_n)P(B_n)=\sum_{j=1}^nP(A|B_j)P(B_j)$$

**贝叶斯定理**：设试验$E$的样本空间为$S$，$A$为$E$的事件，$B_1,B_2,\cdots, B_n$为样本空间$S$的一个划分，且$P(A)>0,P(B_i)\geq 0 (i=1,2,\cdots,n)$，则有：
$$P(B_i|A)=\frac{P(A|B_i)P(B_i)}{\sum_{j=1}^nP(A|B_j)P(B_j)}$$

朴素贝叶斯法假设：在分类正确的条件下，用于分类的特征是条件独立的，即
$$P(X=\vec{x}|Y=c_k)=P(X^{(1)}=x^{(1)},X^{(2)}=x^{(2)},\cdots,X^{(n)}=x^{(n)}|Y=c_k)=\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k),k=1,2,\cdots,K$$

根据贝叶斯定理，
$$P(Y=c_k|X=\vec{x})=\frac{P(X=\vec{x}|Y=c_k)P(Y=c_k)}{\sum_{j=1}^KP(X=\vec{x}|Y=c_j)P(Y=c_j)}$$

考虑分类特征的条件独立假设，有：
$$P(Y=c_k|X=\vec{x})=\frac{P(Y=c_k)\prod_{i=1}^nP(X^{(i)}|Y=c_k)}{\sum_{j=1}^KP(X=\vec{x}|Y=c_j)P(Y=c_j)},k=1,2,\cdots,K$$

对于所有的$c_k$，上式的分母都相同，则得：
$$y=f(\vec{x})=arg\ \underset{c_k}{max}\ P(Y=c_k)\prod_{j=1}^nP(X^{(j)}=x^{(j)}|Y=c_k)$$

先验概率$P(Y=c_k)$的极大似然估计为：
$$P(Y=c_k)=\frac{1}{N}\sum_{i=1}^NI(y_i=c_k),k=1,2,\cdots,K$$

条件概率$P(X^{(j)}=x^{(j)}|Y=c_k)$的**极大似然估计**为：
$$P(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x^{(j)}_i=a_{jl},y_i=c_k)}{\sum_{i=1}^NI(y_i=c_k)}$$
其中，$a_{j1},a_{j2},\cdots,a_{js_j}$为第$j$个特征$x^{(j)}$可能的取值。

_____
用极大似然估计求$P(X^{(j)}=x^{(j)}|Y=c_k)$可能会 出现分母为0的情况。此时，可采用**贝叶斯估计**，等价于在$X^{(j)}$的各个取值的频数上赋予了一个正数$\lambda$：
$$P_{\lambda}(X^{(j)}=a_{jl}|Y=c_k)=\frac{\sum_{i=1}^NI(x^{(j)}_i=a_{jl},y_i=c_k)+\lambda}{\sum_{i=1}^NI(y_i=c_k)+s_j\lambda}$$
此时，$P(Y=c_k)$的贝叶斯估计为：
$$P_{\lambda}(Y=c_k)=\frac{\sum_{i=1}^NI(y_i=c_k)+\lambda}{N+K\lambda}$$
当$\lambda=1$时，为拉普拉斯平滑。
____

scikit中不同的朴素贝叶斯分类器，在于假设了不同的$P(X^{(j)}|y=c_k)$分布。

In [1]:
from sklearn import datasets, naive_bayes
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def show_digits():
    digits = datasets.load_digits()
    fig = plt.figure()
    print("vector from image 0:", digits.data[0])
    for i in range(25):
        ax = fig.add_subplot(5, 5, i+1)
        ax.imshow(digits.images[i], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()

In [3]:
def load_data():
    digits = datasets.load_digits()
    return train_test_split(digits.data, digits.target, test_size=0.25, random_state=0)

____
**GaussianNB**：高斯贝叶斯分类器，假设特征的条件概率分布满足高斯分布。

**sklearn.naive_bayes.GaussianNB(priors=None, var_smoothing=1e-09)**
> 1. 没有参数，不需要调参。

属性：
> 1. class_prior_：(n_classes, )数组，是每个类别的概率。
> 2. class_count_：(n_classes, )数组，是每个类别包含的训练样本数量。
> 3. theta_：(n_classes, n_features)数组，是每个类别上每个特征的均值。
> 4. sigma_：(n_classes, n_features)数组，是每个类别上每个特征的标准差。

方法：
> 1. fit(X, y[, sample_weight])
> 2. partial_fit(X, y[, classes, sample_weight])：追加训练模型。主要用于大规模数据集的训练。将大数据集划分成若干个小数据集。
> 3. predict(X)
> 4. predict_log_proba(X)：返回一个数组，数组的元素依次是X预测为各个类别的概率的对数值。
> 5. predict_proba(X)
> 6. score(X, y[, sample_weight])

In [15]:
def test_GaussianNB(*data):
    X_train, X_test, y_train, y_test = data
    cls = naive_bayes.GaussianNB()
    cls.fit(X_train, y_train)
    print('Training Score: %.2f' % cls.score(X_test, y_test))
    print('Testing Score: %.2f' % cls.score(X_test, y_test))

In [16]:
X_train, X_test, y_train, y_test = load_data()
test_GaussianNB(X_train, X_test, y_train, y_test)

Training Score: 0.83
Testing Score: 0.83


_____
**多项式贝叶斯分类器**：
$$P(X^{(j)}=a_{s_j}|y=c_k)=\frac{N_{kj}+\alpha}{N_k+\alpha n}$$

随着$\alpha$的增长，预测准确率在下降。当$\alpha \rightarrow \infty $时，$\frac{N_{kj}+\alpha}{N_k+\alpha n}\rightarrow \frac{1}{n}$，即对任何类型的特征、该类型特征的任意取值，出现的概率都是$\frac{1}{n}$。

**sklearn.naive_bayes.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)**
> 1. fit_prior：布尔值。如果为True，则不去学习$P(y=c_k)$，替代以均匀分布。
> 2. class_prior：数组。指定了每个分类的先验概率$P(y=c_1),P(y=c_2),\cdots,P(y=c_K)$。

属性：
> 1. class_log_prior_：(n_classes, )数组，给出每个类别调整后的经验概率分布的对数值。
> 2. feature_log_prob_：(n_classes, n_features)数组，给出$P(X^{(j)}|y=c_k)$的经验概率分布的对数值。
> 3. class_count_：(n_classes, )数组，是每个类别包含的训练样本数量。
> 4. feature_count_：(n_classes, n_features)数组。训练过程中，每个类别每个特征遇到的样本数。

方法：
> 1. fit(X, y[, sample_weight])
> 2. partial_fit(X, y[, classes, sample_weight])：追加训练模型。主要用于大规模数据集的训练。将大数据集划分成若干个小数据集。
> 3. predict(X)
> 4. predict_log_proba(X)：返回一个数组，数组的元素依次是X预测为各个类别的概率的对数值。
> 5. predict_proba(X)
> 6. score(X, y[, sample_weight])

In [18]:
def test_MultinomialNB(*data):
    X_train, X_test, y_train, y_test = data
    cls = naive_bayes.MultinomialNB()
    cls.fit(X_train, y_train)
    print('Training Score: %.2f' % cls.score(X_test, y_test))
    print('Testing Score: %.2f' % cls.score(X_test, y_test))
    
X_train, X_test, y_train, y_test = load_data()
test_MultinomialNB(X_train, X_test, y_train, y_test)

Training Score: 0.91
Testing Score: 0.91


_____
**伯努利贝叶斯分类器**：假设特征的条件概率分布满足二项分布。
$$P(X^{(j)}|y=c_k)=pX^{(j)}+(1-p)(1-X^{(j)})$$
其中，$X^{(j)}\in \{0,1\}$，且$P(X^{(j)}=1|y=c_k)=p$。

**sklearn.naive_bayes.BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)**
> 1. binarize：浮点数或None
    - 若为None，假定原始数据已经二元化了。
    - 若为浮点数，会以该数为界，特征取值大于它的作为1；小于它的作为0。
    - 作为一个经验值，binarize可取(所有特征的所有值的最小值+所有特征的所有值的最大值)/2。

属性：
> 1. class_log_prior_：(n_classes, )数组，给出每个类别调整后的经验概率分布的对数值。
> 2. feature_log_prob_：(n_classes, n_features)数组，给出$P(X^{(j)}|y=c_k)$的经验概率分布的对数值。
> 3. class_count_：(n_classes, )数组，是每个类别包含的训练样本数量。
> 4. feature_count_：(n_classes, n_features)数组。训练过程中，每个类别每个特征遇到的样本数。

方法：
> 1. fit(X, y[, sample_weight])
> 2. partial_fit(X, y[, classes, sample_weight])：追加训练模型。主要用于大规模数据集的训练。将大数据集划分成若干个小数据集。
    - classes：(n_classes, )数组，列出了所有可能的类别。第一次调用时，必须传入该参数，后续调用不必传入该参数。
    - sample_weight：(n_samples, )数组。给出每个样本的权重。如果未指定，则全为1。
> 3. predict(X)
> 4. predict_log_proba(X)：返回一个数组，数组的元素依次是X预测为各个类别的概率的对数值。
> 5. predict_proba(X)
> 6. score(X, y[, sample_weight])

In [19]:
def test_BernoulliNB(*data):
    X_train, X_test, y_train, y_test = data
    cls = naive_bayes.BernoulliNB()
    cls.fit(X_train, y_train)
    print('Training Score: %.2f' % cls.score(X_test, y_test))
    print('Testing Score: %.2f' % cls.score(X_test, y_test))
    
X_train, X_test, y_train, y_test = load_data()
test_BernoulliNB(X_train, X_test, y_train, y_test)

Training Score: 0.85
Testing Score: 0.85
